<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=847ffe83edc77ef40fda6ba36edfbfe6072da9708fc89aa3d48166db55835bfd
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-04 15:00:08
-------------------
qualified stocks: 89
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  7.27 L
Current:  1.40 C
-------------------
Today PnL: -8.64 K (-0.06%)
Current PnL: -18.02 L (-12.12%)
CY Booked + Current PnL: -8.16 L (-5.49%)
-------------------
Total profit:  2.91 L
Total loss:  -20.93 L
-------------------
Total Booked + Current PnL: 18.82 L (15.46%)
Total Booked PnL: 36.84 L (30.27%)
Curr Year Booked PnL: 9.86 L (7.04%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 78.75 L (56.24%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,8285.00,-12.98,64.0,H-LC,9.30,173888.0,17711.0,8381.0,2.17,11.34,4.82,16.71,22.0,2.11,1.31,30.76,X40N,BTT,HEALTHCARE
25,DIXON,18940.15,-28.66,70.0,H-MC,7.15,142728.0,27792.0,8792.0,0.46,24.18,6.16,31.83,81.0,3.16,1.08,47.94,X40N,ATH,IT
79,TTKPRESTIG,770.00,105.34,66.0,M-SC,6.66,88504.0,-12273.0,12364.0,-0.92,-12.18,13.97,0.09,245.0,-0.99,0.67,16.27,OX40N,NTT,DURABLES
64,SBIN,863.00,-13.25,48.0,M-LC,4.48,214518.0,13038.0,14180.0,-0.33,6.47,6.61,13.51,60.0,0.92,1.62,19.88,XY25,NTT,BANKS
43,ITC,452.00,-37.08,58.0,H-LC,2.98,203326.0,3188.0,17710.0,0.98,1.59,8.71,10.44,4.0,0.18,1.53,7.25,X40,NTT,FMCG


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,ICICIPRULI,790.00,-21.00,38.0,H-MC,2.18,135882.0,94.0,42653.0,-1.33,0.07,31.39,31.48,107.0,0.00,1.02,12.20,X40,ATH,INSURANCE
31,HAVELLS,2069.16,-0.23,61.0,H-MC,2.04,247244.0,-487.0,75533.0,1.20,-0.20,30.55,30.30,92.0,-0.01,1.86,13.07,X40,ATH,ELECTRICAL
88,WIPRO,420.00,-13.34,43.0,M-LC,6.98,152067.0,1122.0,108333.0,-1.72,0.74,71.24,72.51,53.0,0.01,1.15,6.81,XR,NTT,IT


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HCLTECH,1943.91,-0.65,37.0,H-LC,9.91,226755.0,-15169.0,78435.0,-1.49,-6.27,34.59,26.15,8.0,-0.19,1.71,7.15,X40,ATH,IT
45,JIOFIN,387.00,-2.04,39.0,H-LC,13.94,216615.0,-791.0,54284.0,-1.35,-0.36,25.06,24.61,48.0,-0.01,1.63,54.25,XY24,BTT,FINANCE
58,RELIANCE,1533.00,-15.17,40.0,H-LC,7.51,212456.0,2090.0,26684.0,-0.78,0.99,12.56,13.68,37.0,0.08,1.60,17.68,XY25,NTT,REFINERIES
41,INFY,2275.00,-19.96,43.0,H-LC,11.09,311981.0,-907.0,172588.0,-0.97,-0.29,55.32,54.87,3.0,-0.01,2.35,6.34,X40,BTT,IT
85,VBL,671.64,-13.12,44.0,H-LC,7.17,312843.0,-2999.0,115658.0,-2.95,-0.95,36.97,35.67,5.0,-0.03,2.36,12.77,X40N,ATH,FMCG


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,LTIM,7230.20,-5.49,54.0,H-LC,2.12,234495.0,-11656.0,90867.0,0.39,-4.74,38.75,32.18,16.0,-0.13,1.77,29.60,X200,ATH,IT
52,NESTLEIND,1377.00,-7.71,64.0,H-LC,2.73,284076.0,18650.0,38151.0,1.61,7.03,13.43,21.40,11.0,0.49,2.14,14.87,XY25,NTT,FMCG
43,ITC,452.00,-37.08,58.0,H-LC,2.98,203326.0,3188.0,17710.0,0.98,1.59,8.71,10.44,4.0,0.18,1.53,7.25,X40,NTT,FMCG
19,CIPLA,1795.00,-19.09,58.0,H-LC,5.39,216145.0,11645.0,29763.0,-0.04,5.69,13.77,20.25,10.0,0.39,1.63,14.92,X40N,BTT,PHARMA
85,VBL,671.64,-13.12,44.0,H-LC,7.17,312843.0,-2999.0,115658.0,-2.95,-0.95,36.97,35.67,5.0,-0.03,2.36,12.77,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ATULAUTO,844.00,3737.84,73.0,M-SC,10.45,119581.0,-27787.0,79605.0,16.54,-18.86,66.57,35.16,236.0,-0.35,0.90,22.14,XY24,NTT,AUTO
12,BATAINDIA,2096.00,-31.01,67.0,M-SC,10.11,103053.0,-25617.0,70911.0,6.81,-19.91,68.81,35.20,219.0,-0.36,0.78,18.28,X40,NTT,FOOTWEAR
22,COLPAL,3767.14,5.55,67.0,H-MC,9.10,192332.0,-19409.0,101513.0,3.52,-9.17,52.78,38.77,84.0,-0.19,1.45,14.48,XY25,ATH,FMCG
36,ICICIGI,2260.25,-19.41,45.0,H-MC,4.32,138491.0,2358.0,28765.0,2.72,1.73,20.77,22.86,91.0,0.08,1.04,14.56,X40,ATH,INSURANCE
4,APOLLOHOSP,8285.00,-12.98,64.0,H-LC,9.30,173888.0,17711.0,8381.0,2.17,11.34,4.82,16.71,22.0,2.11,1.31,30.76,X40N,BTT,HEALTHCARE


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,62.0,L-SC,35.21,89030.0,-24519.0,64573.0,-3.42,-21.59,72.53,35.27,268.0,-0.38,0.67,117.90,XR,NTT,HOTELS
85,VBL,671.64,-13.12,44.0,H-LC,7.17,312843.0,-2999.0,115658.0,-2.95,-0.95,36.97,35.67,5.0,-0.03,2.36,12.77,X40N,ATH,FMCG
69,SONACOMS,1006.00,-38.41,41.0,M-SC,13.02,76615.0,-24544.0,99439.0,-2.71,-24.26,129.79,74.03,202.0,-0.25,0.58,8.03,AR,BTT,AUTO
82,UNITDSPR,1644.00,-13.37,48.0,H-LC,8.54,174398.0,-7361.0,42605.0,-1.92,-4.05,24.43,19.39,86.0,-0.17,1.31,3.77,X40N,NTT,BREWERIES
88,WIPRO,420.00,-13.34,43.0,M-LC,6.98,152067.0,1122.0,108333.0,-1.72,0.74,71.24,72.51,53.0,0.01,1.15,6.81,XR,NTT,IT


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,105.34,66.0,M-SC,6.66,88504.0,-12273.0,12364.0,-0.92,-12.18,13.97,0.09,245.0,-0.99,0.67,16.27,OX40N,NTT,DURABLES
80,UJJIVANSFB,60.0,84.36,46.0,H-SC,13.51,116721.0,-25758.0,45276.0,0.42,-18.08,38.79,13.70,163.0,-0.57,0.88,37.76,OX40N,NTT,BANKS
39,INDIGOPNTS,1408.0,117.63,54.0,M-SC,3.29,142724.0,-31835.0,31870.0,0.71,-18.24,22.33,0.02,221.0,-1.00,1.08,24.15,OX40N,NTT,PAINTS
46,KANSAINER,340.0,-68.34,53.0,H-SC,3.67,218304.0,-51363.0,87693.0,-0.82,-19.05,40.17,13.47,138.0,-0.59,1.64,10.42,XY24,NTT,PAINTS
18,CERA,9475.0,-19.57,47.0,H-SC,2.90,114777.0,-29026.0,55770.0,1.09,-20.18,48.59,18.60,149.0,-0.52,0.86,26.66,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4998.00,-27.08,48.0,H-LC,14.51,290996.0,-54960.0,178817.0,-0.06,-15.89,61.45,35.80,1.0,-0.31,2.19,3.09,X40,BTT,IT
41,INFY,2275.00,-19.96,43.0,H-LC,11.09,311981.0,-907.0,172588.0,-0.97,-0.29,55.32,54.87,3.0,-0.01,2.35,6.34,X40,BTT,IT
43,ITC,452.00,-37.08,58.0,H-LC,2.98,203326.0,3188.0,17710.0,0.98,1.59,8.71,10.44,4.0,0.18,1.53,7.25,X40,NTT,FMCG
85,VBL,671.64,-13.12,44.0,H-LC,7.17,312843.0,-2999.0,115658.0,-2.95,-0.95,36.97,35.67,5.0,-0.03,2.36,12.77,X40N,ATH,FMCG
1,ABB,7934.00,-40.64,48.0,H-LC,13.24,248174.0,-13445.0,132649.0,-0.35,-5.14,53.45,45.57,7.0,-0.10,1.87,5.60,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187495.0,36582.0,20718.0,0.15,24.24,11.05,37.97,93.0,1.77,1.41,55.68,AR,ATH,AUTO
25,DIXON,18940.15,-28.66,70.0,H-MC,7.15,142728.0,27792.0,8792.0,0.46,24.18,6.16,31.83,81.0,3.16,1.08,47.94,X40N,ATH,IT
26,DMART,5201.00,-6.62,70.0,H-LC,14.38,233166.0,48685.0,21684.0,1.68,26.39,9.30,38.14,38.0,2.25,1.76,39.79,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,12.69,64.0,H-MC,3.37,194610.0,29940.0,38027.0,-0.74,18.18,19.54,41.28,89.0,0.79,1.47,27.62,X40N,ATH,AC
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187495.0,36582.0,20718.0,0.15,24.24,11.05,37.97,93.0,1.77,1.41,55.68,AR,ATH,AUTO
86,VOLTAS,1918.49,0.74,61.0,H-MC,4.56,211305.0,19563.0,76471.0,-1.51,10.20,36.19,50.08,99.0,0.26,1.59,17.69,XY25,ATH,AC
23,DABUR,735.00,-0.95,68.0,H-MC,4.33,215416.0,18388.0,71238.0,1.63,9.33,33.07,45.49,102.0,0.26,1.62,23.59,XY24,BTT,FMCG
17,CAMS,5211.76,-7.16,45.0,H-SC,5.57,105504.0,3500.0,40429.0,-0.94,3.43,38.32,43.06,122.0,0.09,0.79,21.13,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187495.0,36582.0,20718.0,0.15,24.24,11.05,37.97,93.0,1.77,1.41,55.68,AR,ATH,AUTO
51,NATIONALUM,247.44,-45.32,71.0,H-MC,0.23,109630.0,9126.0,21509.0,-0.13,9.08,19.62,30.49,79.0,0.42,0.83,44.43,MH,ATH,METALS
87,WHIRLPOOL,2270.00,-40.03,59.0,M-SC,4.66,101272.0,9774.0,68976.0,0.40,10.68,68.11,86.07,223.0,0.14,0.76,47.07,XR,NTT,DURABLES
38,INDIAMART,4850.92,-51.47,47.0,H-SC,12.50,135712.0,12376.0,121381.0,1.30,10.03,89.44,108.45,119.0,0.10,1.02,34.63,AR,ATH,MISC
88,WIPRO,420.00,-13.34,43.0,M-LC,6.98,152067.0,1122.0,108333.0,-1.72,0.74,71.24,72.51,53.0,0.01,1.15,6.81,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SIS,528.00,2233.16,53.0,H-SC,5.74,62772.0,-18318.0,26986.0,-0.79,-22.59,42.99,10.69,156.0,-0.68,0.47,27.22,OX40N,NTT,MISC
21,COFFEEDAY,80.00,-39.64,62.0,L-SC,35.21,89030.0,-24519.0,64573.0,-3.42,-21.59,72.53,35.27,268.0,-0.38,0.67,117.90,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-3.16,48.0,H-SC,15.91,90192.0,-10771.0,105299.0,-0.54,-10.67,116.75,93.63,148.0,-0.10,0.68,34.34,SR,ATH,CHEMICALS
50,MASFIN,398.61,-17.80,52.0,H-SC,13.27,93285.0,-4695.0,26297.0,-0.50,-4.79,28.19,22.05,152.0,-0.18,0.70,35.60,XR,ATH,FINANCE
51,NATIONALUM,247.44,-45.32,71.0,H-MC,0.23,109630.0,9126.0,21509.0,-0.13,9.08,19.62,30.49,79.0,0.42,0.83,44.43,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-39.64,62.0,L-SC,35.21,89030.0,-24519.0,64573.0,-3.42,-21.59,72.53,35.27,268.0,-0.38,0.67,117.90,XR,NTT,HOTELS
33,HEROMOTOCO,5949.07,1.28,78.0,H-MC,20.37,187495.0,36582.0,20718.0,0.15,24.24,11.05,37.97,93.0,1.77,1.41,55.68,AR,ATH,AUTO
26,DMART,5201.00,-6.62,70.0,H-LC,14.38,233166.0,48685.0,21684.0,1.68,26.39,9.30,38.14,38.0,2.25,1.76,39.79,X40N,NTT,FMCG
57,RELAXO,1176.00,-30.87,64.0,H-SC,5.44,64128.0,-51410.0,86400.0,0.32,-44.50,134.73,30.28,136.0,-0.60,0.48,26.08,X40N,NTT,FOOTWEAR
7,ATULAUTO,844.00,3737.84,73.0,M-SC,10.45,119581.0,-27787.0,79605.0,16.54,-18.86,66.57,35.16,236.0,-0.35,0.90,22.14,XY24,NTT,AUTO


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.89
1,25,43.88
2,50,73.95


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    37.80
LC    36.32
MC    25.85
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.68
X40      15.71
X40N     14.39
XY25     10.11
AR        9.50
XR        9.04
OX40N     7.46
X200      1.77
MH        1.76
X5K       1.44
SR        1.11
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    28.05
H-MC    22.86
H-SC    22.56
M-SC    13.65
M-LC     7.18
M-MC     2.67
L-SC     1.59
L-LC     1.09
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,1.08,32.94
IT,13.47,-15.55,72.84
FINANCE,7.39,-22.69,67.93
BANKS,7.38,-16.52,66.00
PAINTS,6.08,-14.36,39.96
MISC,5.84,-18.04,85.71
ELECTRICAL,5.74,-8.01,67.54
AUTO,4.77,-6.70,53.13
AC,3.79,3.78,28.81


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2787566.0,22
AR,1217470.0,10
XR,1091963.0,13
X40,880833.0,10
X40N,524059.0,11
OX40N,467211.0,10
XY25,409714.0,7
SR,195943.0,2
X5K,126928.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2775419.0,27
M-SC,1433264.0,17
H-LC,1396092.0,16
H-MC,1170606.0,17
M-LC,427458.0,5
M-MC,322499.0,2
L-SC,250400.0,3
L-MC,59540.0,1
L-LC,39525.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,933059.0,6
M-SC,XY24,848547.0,7
H-SC,AR,662591.0,5
H-LC,X40,595212.0,5
H-SC,XR,525166.0,6
H-MC,XY24,451160.0,4
H-LC,AR,372803.0,2
M-MC,XY24,322499.0,2
H-LC,X40N,218091.0,5


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
